# PT03 - Die MNIST-DB und PyTorch

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from mnist import MnistDataset

%matplotlib inline

ModuleNotFoundError: No module named 'mnist'

### Warnungen ausschalten

In [2]:
import warnings
warnings.filterwarnings("ignore")

### GPU-Check

In [ ]:
device =  "cuda" if torch.cuda.is_available else "cpu"

### MNIST-Datensatz einlesen

In [5]:
from pathlib import Path
from sklearn.datasets import fetch_openml

def load_mnist():
    if not Path('mnist_data.csv').exists():
        mnist = fetch_openml('mnist_784', version=1, parser="auto")
        mnist.data.to_csv("mnist_data.csv",index=False)
        mnist.target.to_csv("mnist_target.csv",index=False)
        return mnist.data, mnist.target.astype(int)
    else:
        return pd.read_csv('mnist_data.csv'), pd.read_csv('mnist_target.csv').astype(int)

X, y = load_mnist()        

### Die Klasse Classifier

In [8]:
class Classifier(nn.Module):    
   
    def __init__(self):
        super().__init__()
        
        self.model = nn.Sequential(
            nn.Linear(784,200),
            nn.Sigmoid(),
            nn.Linear(200,10),
            nn.Sigmoid()     
        )
        self.loss_function = nn.MSELoss()
        self.optimiser = torch.optim.SGD(self.parameters(),lr=0.01)
        
        self.counter = 0
        self.progress = []
        
    def forward(self,inputs):
        return self.model(inputs)
    
    def train(self,inputs,targets):
        outputs = self.forward(inputs)
        loss = self.loss_function(outputs,targets)
        self.optimiser.zero_grad()
        loss.backward()
        self.optimiser.step()
        
        self.counter += 1
        if (self.counter % 10) == 0:
            self.progress.append(loss.item())
        
        if (self.counter % 10000) == 0:
            print(f'Counter = {self.counter}')
        
    def plot_progress(self):
        df = pd.DataFrame(self.progress,columns = ['loss'])
        df.plot(ylim=(0,1.0),figsize=(16,8),alpha=0.1,marker='.',grid=True, yticks=(0,0.25,0.5))
        
        

#### Training des Netzes

In [9]:
C = Classifier()

for label, image_data_tensor, target_tensor in mnist_ds:
    C.train(image_data_tensor,target_tensor)
 

NameError: name 'mnist_ds' is not defined

#### Test des Netzes

In [ ]:
mnist_test_ds = MnistDataset('daten/mnist_test.csv')

In [ ]:
record = 19
#mnist_test_ds.plot_image(record)

In [ ]:
image_data = mnist_test_ds[record][1]

output = C.forward(image_data)

pd.DataFrame(output.detach().numpy()).plot(kind='bar',legend='False',ylim=(0,1))